In [1]:
import pandas as pd
train_2022 = pd.read_csv('train_2022.csv')
data_augmentation_chatGPT = pd.read_csv('data_augmentation_chatGPT.csv')
data_augmentation_random_2_words = pd.read_csv('data_augmentation_random_2_words.csv')
data_augmentation_random_3_words = pd.read_csv('data_augmentation_random_3_words.csv')
translated_en_data = pd.read_csv('translated_en_data.csv')

In [2]:
merged_random_data = pd.concat([data_augmentation_random_2_words, data_augmentation_random_3_words], ignore_index=True)
merged_random_data = merged_random_data.sample(n=2000, random_state=42)
merged_random_data = merged_random_data.reset_index(drop=True)
# merged_random_data

In [3]:
merged_data = pd.concat([train_2022, merged_random_data,translated_en_data, data_augmentation_chatGPT], ignore_index=True)
merged_data

,row_id,TEXT,LABEL
0,0,director dirk shafer and co-writer greg hinton...,0
1,1,"a charming , quirky and leisurely paced scotti...",1
2,2,"the price was good , and came quickly though ...",1
3,3,i was looking forward to this game for a coupl...,0
4,4,arguably the year 's silliest and most incoher...,0
...,...,...,...
7995,1995,A creative comedy/thriller.,1
7996,1996,Explores paranoia and insecurity in America's ...,1
7997,1997,Good for power grating.,1
7998,1998,McGrath's variation on the novel crafts moving...,1


In [4]:
# %%time

# from transformers import BertTokenizer, BertForSequenceClassification, AdamW
# from torch.utils.data import DataLoader, TensorDataset
# import torch
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# from torch.nn.utils.rnn import pad_sequence

# # 載入預訓練的 BERT tokenizer 和模型
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=2)  # 2 表示二分類，正向和負向情感

# # 轉換成 DataFrame
# train_data = merged_data.copy()

# # 使用 tokenizer 將文本轉換為 token IDs
# def tokenize_text(text):
#     return tokenizer(text, return_tensors='pt')['input_ids'][0]

# train_data['input_ids'] = train_data['TEXT'].apply(tokenize_text)

# # 轉換成可以輸入模型的格式
# inputs = pad_sequence(train_data['input_ids'].tolist(), batch_first=True)

# # 將資料拆分為訓練集和測試集
# train_inputs, test_inputs, train_labels, test_labels = train_test_split(inputs, train_data['LABEL'].tolist(), test_size=0.2)

# # 創建 PyTorch DataLoader
# train_dataset = TensorDataset(train_inputs, torch.tensor(train_labels))
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# # 定義 optimizer 和損失函數
# optimizer = AdamW(model.parameters(), lr=1e-5)
# loss_fn = torch.nn.CrossEntropyLoss()

# # 訓練模型
# model.train()
# for epoch in range(5):
#     for inputs, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = loss_fn(outputs.logits, labels)
#         loss.backward()
#         optimizer.step()

# # 評估模型
# model.eval()
# with torch.no_grad():
#     outputs = model(test_inputs)
#     predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

# # 計算分類報告
# report = classification_report(test_labels, predicted_labels)
# print(report)

In [4]:
import torch

# 檢查CUDA是否可用
cuda_available = torch.cuda.is_available()

if cuda_available:
    print("CUDA is available.")
else:
    print("CUDA is not available.")

CUDA is available.


# cuda

In [ ]:
%%time

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.nn.utils.rnn import pad_sequence

# 確認是否有可用的 CUDA 設備，並設定使用的設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 載入預訓練的 BERT tokenizer 和模型，並將它們移動到 CUDA 設備上
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=2)
model.to(device)

# 轉換成 DataFrame
train_data = merged_data.copy()

# 使用 tokenizer 將文本轉換為 token IDs
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt')['input_ids'][0]

train_data['input_ids'] = train_data['TEXT'].apply(tokenize_text)

# 轉換成可以輸入模型的格式，並移動到 CUDA 設備上
inputs = pad_sequence(train_data['input_ids'].tolist(), batch_first=True).to(device)

# 將資料拆分為訓練集和測試集
train_inputs, test_inputs, train_labels, test_labels = train_test_split(inputs, train_data['LABEL'].tolist(), test_size=0.2, random_state=42)

# 創建 PyTorch DataLoader，並設定為使用 CUDA
train_dataset = TensorDataset(train_inputs, torch.tensor(train_labels).to(device))
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)

# 定義 optimizer 和損失函數
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# 訓練模型
model.train()
for epoch in range(5):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)  # 將資料移動到 CUDA 設備上
        outputs = model(inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

# 評估模型
model.eval()
with torch.no_grad():
    test_inputs = test_inputs.to(device)  # 將測試資料移動到 CUDA 設備上
    outputs = model(test_inputs)
    predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

# 計算分類報告
report = classification_report(test_labels, predicted_labels)
print(report)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


              precision    recall  f1-score   support

           0       0.50      1.00      0.66       797
           1       0.00      0.00      0.00       803

    accuracy                           0.50      1600
   macro avg       0.25      0.50      0.33      1600
weighted avg       0.25      0.50      0.33      1600

CPU times: total: 11min 11s
Wall time: 20min 33s


D:\Program Files\anaconda3\envs\Pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program Files\anaconda3\envs\Pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program Files\anaconda3\envs\Pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
import datetime
import pytz
import pandas as pd
def export_csv(df,name):
  now = datetime.datetime.now().astimezone(pytz.timezone('Asia/Taipei'))
  formatted_time = now.strftime('%Y%m%d')
  df.to_csv('result/'+ formatted_time + '_' + name + ".csv", index=False,encoding="utf_8_sig")

In [8]:
%%time
# 設定 device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 載入預測資料集
test_data = pd.read_csv('test_no_answer_2022.csv')

# 使用 tokenizer 將文本轉換為 token IDs
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt')['input_ids'][0]

test_data['input_ids'] = test_data['TEXT'].apply(tokenize_text)

# 轉換成可以輸入模型的格式
test_inputs = pad_sequence(test_data['input_ids'].tolist(), batch_first=True).to(device)

# 創建 PyTorch DataLoader
test_dataset = TensorDataset(test_inputs)
test_loader = DataLoader(test_dataset, batch_size=8)

# 使用模型進行預測
model.eval()
predictions = []
with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs[0])  # 確保 inputs[0] 已在 GPU 上
        predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()
        predictions.extend(predicted_labels)

# 將預測結果添加到測試數據集中
test_data['LABEL'] = predictions
# 保存預測結果到 CSV 文件
# 需要你自己定義 export_csv 函數，或使用 pandas 的 to_csv 方法
export_csv(test_data[['row_id', 'LABEL']], 'BERT_Large_DataAugmentation')
# 打印預測結果
print(test_data[['row_id','TEXT', 'LABEL']])

       row_id                                               TEXT  LABEL
0           0   good to know if you can t find these elsewhere .      0
1           1  love it !  the grill plates come out and pop i...      0
2           2  i m convinced this was a poorly executed refur...      0
3           3  i would never have complained about that if it...      0
4           4  the photo shows the same whole ,  large candie...      0
...       ...                                                ...    ...
10995   10995             i didn t quite get it the first time .      0
10996   10996  i ve tried installing with and without the oem...      0
10997   10997  i was parked at a truck stop in the cincinnati...      0
10998   10998  i recently bought this case after seeing some ...      0
10999   10999  the keyboard types only % of the time and the ...      0

[11000 rows x 3 columns]
CPU times: total: 57.6 s
Wall time: 1min 2s


In [9]:
# %%time
# # 載入預測資料集
# test_data = pd.read_csv('test_no_answer_2022.csv')

# # 使用 tokenizer 將文本轉換為 token IDs
# def tokenize_text(text):
#     return tokenizer(text, return_tensors='pt')['input_ids'][0]

# test_data['input_ids'] = test_data['TEXT'].apply(tokenize_text)

# # 轉換成可以輸入模型的格式
# test_inputs = pad_sequence(test_data['input_ids'].tolist(), batch_first=True)

# # 創建 PyTorch DataLoader
# test_dataset = TensorDataset(test_inputs)
# test_loader = DataLoader(test_dataset, batch_size=8)

# # 使用模型進行預測
# model.eval()
# predictions = []
# with torch.no_grad():
#     for inputs in test_loader:
#         outputs = model(inputs[0])  # inputs[0] 是 token IDs
#         predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()
#         predictions.extend(predicted_labels)

# # 將預測結果添加到測試數據集中
# test_data['LABEL'] = predictions
# # 保存預測結果到 CSV 文件
# export_csv(test_data.drop(columns=['TEXT','input_ids']),'BERT_Large_DataAugmentation')

# # 打印預測結果
# print(test_data[['TEXT', 'LABEL']])

In [10]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118